# Dockerizing our Lambda Function

### Introduction

So far we have worked with various lambda functions.  And we have seen that the code, and the libraries needed to run the code can get fairly complicated.

As our lambda functions become more complicated, we'll want to ensure that our development environment matches that of our lambda environment.  Sounds like a case for Docker.

So in this lesson, we'll see how we can dockerize our lambda function and then run it on AWS.  Let's get started.

### Just running Python

As a first step, we should make sure the code works outside of Docker by just running our Python code.  

So if you look at the `codebase/src/main.py` file, you'll see that it uses pandas to hit an api.  

```bash
python3 -i main.py
```

And from there, we run our function, passing through two dummy arguments.

```python
lambda_handler('', '')
```

### Dockerizing our code

After confirming that the Python code works, it's time to dockerize it.  We have the following Dockerfile to produce an image.

```bash
FROM public.ecr.aws/lambda/python:3.9-arm64

COPY ./src/requirements.txt ./
RUN pip3 install -r requirements.txt
COPY ./src/main.py ./

CMD ["main.lambda_handler"]
```

> You can see more details [in the documentation](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html#images-create-from-base).

Above we use AWS's lambda image as our base image.  Then we install our `requirements.txt` and copy over our `main.py` file into the image.

Finally we specify the command that we want to be run when our Docker image starts.  And this is just the module that holds our lambda function followed by the name of our lambda function.

**One important caveat**: The base image should align with your local computing environment.  So here, we are assuming that we are building our image on a computer with a mac M1 or M2 chip.  However, if you are not, change the base image line to: `FROM public.ecr.aws/lambda/python:3.9`.  That is, remove the `-arm64` flag.

Ok, now let's build our Docker image, and tag it with a name of `get_tx_receipts`.  

```bash
docker build -t get_tx_receipts .
```

Then we can try running our container locally.

```bash
docker run -p 9000:8080 get_tx_receipts:latest
```

So we are running the container, mapping the port to our host port of 9000.  And from there, we can make a request to our lambda funciton by going to a new tab in our terminal and entering the following.

```bash
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d "{\"msg\":\"hello\"}"
```

We should see the same output of a couple of records from our API.

### Deploying to AWS

Ok, now that we can see our container working in docker, the next step is to push our image to AWS, and then to have lambda run this image.

We can push an image to AWS by using the elastic container registry in AWS.  So search for that in the AWS console.

<img src="./ecr.png" width="100%">

Then click on get started, and add a repository name, and click on create repository.  

> This is similar to creating a repository in github.

<img src="./repo-name.png" width="100%">

From here, you can click on the repository you just created and click on `View push commands` to see how to add your docker image to this ECR repository.

> <img src="./view-push.png" width="100%">

The instructions there are quite good, and you just need to follow them line by line (read what they're doing as well.)  When your image is pushed to the repository, you should see it show up under images.

### Creating our lambda function

Ok, now after pushing our image to ECR, we can use that image to run our lambda function.  To do so, go search for lambda in the AWS console, and create a new function.

Select that you are creating an function from a container image, name your function, and then select Browse Images to find the image added to ECR.

<img src="./create-lambda.png" width="80%">

You may have to hit the refresh button to the right, but eventually you should see your ECR image listed.  Select it, and then choose `Select image`.

<img src="./container-img.png" width="80%">

Finally, under architecture, make sure that it matches what was specified in your image.  So if you are on a mac m1, and used a base image of `public.ecr.aws/lambda/python:3.9-arm64`, then you should choose an architecture of `arm64`.

Finally, click create function, and then create a test to event to ensure that your lambda function works. 

* Troubleshooting

If you see something like the following, it's likely because you chose a base image that does not match your coding environment.

<img src="./format-error.png">

So you'll need to ensure the base image matches your environment (only have a base image of `public.ecr.aws/lambda/python:3.9-arm64` if you have an mac m1 or m2), then rebuild the image locally, push again to ECR, and create a new lambda function selecting that image. 

### Development workflow

Going forward, we'll want to be able to update our lambda function.  Because we're using docker this time, our steps will be the following.

* Make the changes locally
* Rebuild our image locally
* Push our image up to AWS ECR
* Select deploy a new image in our lambda function to use that new image

<img src="./deploy-image.png" width="80%">

### Summary

In this lesson, we saw how we can use docker in our lambda function.  This is useful when the dependencies to run our lambda functions increase, as well as the codebase.  Using an image, we are allocated 10 gb of space in our lambda function, so we can install a lot of underlying software to accomplish our task.  And using docker will better ensure that our production environment matches our development environment.

To accomplish this, we used ECR the elastic container registry.  This allowed us to deploy a docker image to AWS, keep that image private, and use it in our lambda function.

### Resources

[Python lambda docker video](https://www.youtube.com/watch?v=2VtuNOEw8S4&t=1s&ab_channel=SoumilShah)

[AWS Documentation](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html#images-create-from-base)